In [1]:
import numpy as np
from math import *
import time, sched
import board
import digitalio
from numpy import interp
from adafruit_servokit import ServoKit
from src import kinematics, joystick
import neopixel_spi as neopixel
kit = ServoKit(channels=16 , frequency=300)

In [2]:
#Servo Calibration
kit.servo[0].set_pulse_width_range(min_pulse=760, max_pulse = 2130) # FL Hip
kit.servo[1].set_pulse_width_range(min_pulse=835, max_pulse = 2185) # FL Knee
kit.servo[2].set_pulse_width_range(min_pulse=870, max_pulse = 2220) # FL Leg
kit.servo[3].set_pulse_width_range(min_pulse=870, max_pulse = 2225) # BL Hip
kit.servo[4].set_pulse_width_range(min_pulse=1005, max_pulse = 2325) # BL Knee
kit.servo[5].set_pulse_width_range(min_pulse=900, max_pulse = 2210) # BL Leg
kit.servo[6].set_pulse_width_range(min_pulse=810, max_pulse = 2180) # BR Hip
kit.servo[7].set_pulse_width_range(min_pulse=785, max_pulse = 2095) # BR Knee
kit.servo[8].set_pulse_width_range(min_pulse=800, max_pulse = 2110) # BR Leg
kit.servo[9].set_pulse_width_range(min_pulse=800, max_pulse = 2170) # FR Hip
kit.servo[10].set_pulse_width_range(min_pulse=910, max_pulse = 2250) # FR Knee
kit.servo[11].set_pulse_width_range(min_pulse=800, max_pulse = 2090)# FR Leg

In [3]:
# Function to write angles to servos
# Interpolate maps out the angle from -90 to 90 degrees to the servo configuriation of 0 to 180. 
# Some servos angles are inversed due to the installation of the servo.

def write_angles(angles):
    # FL
    if angles[0][0] >= -30 and angles[0][0] <= 30:
        kit.servo[0].angle = interp(angles[0][0], [-90, 90], [180, 0]) # Link 1

    if angles[0][1] >= -90 and angles[0][1] <= 30:
        kit.servo[1].angle = interp(angles[0][1], [-90, 90], [180, 0]) # Link 2

    if angles[0][2] >= -55 and angles[0][2] <= 60:
        kit.servo[2].angle = interp(angles[0][2], [-90, 90], [0, 180]) # Link 3

    # BL
    if angles[1][0] >= -30 and angles[1][0] <= 30:
        kit.servo[3].angle = interp(angles[1][0], [-90, 90], [0, 180]) # Link 1

    if angles[1][1] >= -90 and angles[1][1] <= 30:
        kit.servo[4].angle = interp(angles[1][1], [-90, 90], [180, 0]) # Link 2

    if angles[1][2] >= -55 and angles[1][2] <= 60:
        kit.servo[5].angle = interp(angles[1][2], [-90, 90], [0, 180]) # Link 3

    # BR
    if angles[2][0] >= -30 and angles[2][0] <= 30:
        kit.servo[6].angle = interp(angles[2][0], [-90, 90], [180, 0]) # Link 1

    if angles[2][1] >= -90 and angles[2][1] <= 30:
        kit.servo[7].angle = interp(angles[2][1], [-90, 90], [0, 180]) # Link 2

    if angles[2][2] >= -55 and angles[2][2] <= 60:
        kit.servo[8].angle = interp(angles[2][2], [-90, 90], [180, 0]) # Link 3

    # FR
    if angles[3][0] >= -30 and angles[3][0] <= 30:
        kit.servo[9].angle = interp(angles[3][0], [-90, 90], [0, 180]) # Link 1

    if angles[3][1] >= -90 and angles[3][1] <= 30:
        kit.servo[10].angle = interp(angles[3][1], [-90, 90], [0, 180]) # Link 2

    if angles[3][2] >= -55 and angles[3][2] <= 60:
        kit.servo[11].angle = interp(angles[3][2], [-90, 90], [180, 0]) # Link 3
        

In [4]:
# Function for simple trot gait where LY controls forward and backward step length, LX controls side step length and RX controls yaw
# np.array([FL, FR, BL, BR])

def trot_gait(LX, LY, RX, step_height):
    
    T1 = neutral_standing + np.array([[0,step_height,0,0],[0,0,0,0],[0,0,0,0],[0,step_height,0,0]])
    T2 = neutral_standing + np.array([[step_lengthx*LY,0,-step_lengthz*RX-step_lengthz*LX,0],[-step_lengthx*LY,0,step_lengthz*RX+step_lengthz*LX,0],[-step_lengthx*LY,0,-step_lengthz*RX+step_lengthz*LX,0],[step_lengthx*LY,0,step_lengthz*RX-step_lengthz*LX,0]])
    T3 = neutral_standing + np.array([[0,0,0,0],[0,step_height,0,0],[0,step_height,0,0],[0,0,0,0]])
    T4 = neutral_standing + np.array([[-step_lengthx*LY,0,step_lengthz*RX+step_lengthz*LX,0],[step_lengthx*LY,0,-step_lengthz*RX-step_lengthz*LX,0],[step_lengthx*LY,0,step_lengthz*RX-step_lengthz*LX,0],[-step_lengthx*LY,0,-step_lengthz*RX+step_lengthz*LX,0]])
    
    return [T1,T2,T3,T4]

def walk_gait(LX, LY, RX, step_height):
    
    W1 = neutral_standing + np.array([[-step_lengthx*LY,0,step_lengthz*RX+step_lengthz*LX,0],[(1/3)*step_lengthx*LY,0,(1/3)*(-step_lengthz*RX-step_lengthz*LX),0],[-(1/3)*step_lengthx*LY,0,(1/3)*(-step_lengthz*RX+step_lengthz*LX),0],[step_lengthx*LY,0,step_lengthz*RX-step_lengthz*LX,0]])
    W2 = neutral_standing + np.array([[0,step_height,0,0],[(1/3)*step_lengthx*LY,0,(1/3)*(-step_lengthz*RX-step_lengthz*LX),0],[-(1/3)*step_lengthx*LY,0,(1/3)*(-step_lengthz*RX+step_lengthz*LX),0],[step_lengthx*LY,0,step_lengthz*RX-step_lengthz*LX,0]])
    W3 = neutral_standing + np.array([[step_lengthx*LY,0,-step_lengthz*RX-step_lengthz*LX,0],[-(1/3)*step_lengthx*LY,0,(1/3)*(step_lengthz*RX+step_lengthz*LX),0],[-step_lengthx*LY,0,-step_lengthz*RX+step_lengthz*LX,0],[(1/3)*step_lengthx*LY,0,(1/3)*(step_lengthz*RX-step_lengthz*LX),0]])
    W4 = neutral_standing + np.array([[step_lengthx*LY,0,-step_lengthz*RX-step_lengthz*LX,0],[-(1/3)*step_lengthx*LY,0,(1/3)*(step_lengthz*RX+step_lengthz*LX),0],[0,step_height,0,0],[(1/3)*step_lengthx*LY,0,(1/3)*(step_lengthz*RX-step_lengthz*LX),0]])
    W5 = neutral_standing + np.array([[(1/3)*step_lengthx*LY,0,(1/3)*(-step_lengthz*RX-step_lengthz*LX),0],[-step_lengthx*LY,0,step_lengthz*RX+step_lengthz*LX,0],[step_lengthx*LY,0,step_lengthz*RX-step_lengthz*LX,0],[-(1/3)*step_lengthx*LY,0,(1/3)*(-step_lengthz*RX+step_lengthz*LX),0]])   
    W6 = neutral_standing + np.array([[(1/3)*step_lengthx*LY,0,(1/3)*(-step_lengthz*RX-step_lengthz*LX),0],[0,step_height,0,0],[step_lengthx*LY,0,step_lengthz*RX-step_lengthz*LX,0],[-(1/3)*step_lengthx*LY,0,(1/3)*(-step_lengthz*RX+step_lengthz*LX),0]])
    W7 = neutral_standing + np.array([[-(1/3)*step_lengthx*LY,0,(1/3)*(step_lengthz*RX+step_lengthz*LX),0],[step_lengthx*LY,0,-step_lengthz*RX-step_lengthz*LX,0],[(1/3)*step_lengthx*LY,0,(1/3)*(step_lengthz*RX-step_lengthz*LX),0],[-step_lengthx*LY,0,-step_lengthz*RX+step_lengthz*LX,0]])   
    W8 = neutral_standing + np.array([[-(1/3)*step_lengthx*LY,0,(1/3)*(step_lengthz*RX+step_lengthz*LX),0],[step_lengthx*LY,0,-step_lengthz*RX-step_lengthz*LX,0],[(1/3)*step_lengthx*LY,0,(1/3)*(step_lengthz*RX-step_lengthz*LX),0],[0,step_height,0,0]])
    
    return [W1, W2, W3, W4, W5, W6, W7, W8]

In [5]:
### Initialization and Configuration ###
# --- Arrays for leg position ---
neutral_standing = np.array([[117,-sqrt(200)*10,88.5,1], # np.array([FL, FR, BL, BR])
                             [117,-sqrt(200)*10,-88.5,1],
                             [-117,-sqrt(200)*10,88.5,1],
                             [-117,-sqrt(200)*10,-88.5,1]])
neutral_resting = np.array([[117,-60,88.5,1], 
                             [117,-60,-88.5,1],
                             [-117,-60,88.5,1],
                             [-117,-60,-88.5,1]])
prev_angles = np.array([[0, -50.7, 51.3],   # FL
                   [0, -50.7, 51.3],   # BL
                   [0, -50.7, 51.3],   # BR
                   [0, -50.7, 51.3]])  # FR  
current_angles = prev_angles

# --- Gait Configuration --- 
# X-axis Step Length is moving forward and backwards
# Z-axis Step length is moving sidewards and yawing
step_height = 17
step_lengthx = 0.50
step_lengthz = 0.30
loop_time = 14
max_loop_time = 14
height = 0

# --- Loop Initialization ---
# Timers
tik = int(round(time.time() * 1000))
rest_tik = int(round(time.time() * 1000))
stand_tik = int(round(time.time() * 1000))
kin_tik = int(round(time.time() * 1000))
led_tik = int(round(time.time() * 1000))

i = 0
j = 0
b = joystick.BluetoothController()
flag = 0
loop_time = 14
max_loop_time = 14

# Relay code to cut servo power if needed
# True = Power on
# False = Power off
relay = digitalio.DigitalInOut(board.D17)
relay.direction = digitalio.Direction.OUTPUT
relay.value = True

# --- States ---
# Rest = 0
# Start-Up = 1
# Walking = 2
# I-Kinematics = 3
state = 0
prev_state = 0

# --- NEOPIXEL ---
NUM_PIXELS = 12
N = 0
NA = 6
PIXEL_ORDER = neopixel.GRB
# COLORS in RGB
Red = (255,10,51)
Green = (20, 240, 20)
Blue = (9,120,250)
Orange = (255,80,0)

BRIGHTNESS = 0.5
spi = board.SPI()
 
pixels = neopixel.NeoPixel_SPI(
    spi, NUM_PIXELS, pixel_order=PIXEL_ORDER, auto_write=False, brightness = BRIGHTNESS
)

device /dev/input/event2, name "Pro Controller", phys "00:1a:7d:da:71:15"


In [9]:
# --- Main Loop ---

while(1):
    # ------ Read input from controller ------
    [button, LX, LY, RX, RY] = b.read_controller()
    # Change States
    if button == 'B':
        state = 0
    elif button == 'H':
        state = 1
    elif button == 'A':
        state = 2
    elif button == 'Y':
        state = 3
    # Change Gait Parameters
    if button == '+' and step_height < 17 + 10:
        step_height += 10
    elif button == '-' and step_height > 17 - 10:
        step_height -= 10
    elif button == 'ZL' and loop_time < loop_time + 30:
        loop_time += 10
    elif button == 'ZR' and loop_time >= max_loop_time + 10:
        loop_time -= 10
    elif button == 'L' and height > -25:
        height -= 20
    elif button == 'R' and height < 25:
        height += 20
    
    # STATES
    # --- Rest ---
    if state == 0:
        angles = np.round(kinematics.solve(neutral_resting,kinematics.bodyIK(0,0,0*pi/180,28,0,0)), decimals=1)
        inc_angles = np.round((angles-prev_angles)/50, decimals=1)
        if int(round(time.time() * 1000)) - rest_tik > 30:
            rest_tik = int(round(time.time() * 1000)) 
            if i < 50:
                pixels.fill(Red)
                pixels.show()
                current_angles = current_angles + inc_angles
                write_angles(current_angles)
                i+= 1
            elif i == 50:
                j = 0
                prev_angles = current_angles
                relay.value = False
                prev_state = 0
                pixels.fill(0)
                pixels.show()
    # --- Start Up ---
    if state == 1 and prev_state == 0:
        relay.value = True
        pixels.fill(Green)
        pixels.show()
        angles = np.round(kinematics.solve(neutral_standing,kinematics.bodyIK(0,0,0*pi/180,28,0,0)), decimals=1)
        inc_angles = np.round((angles-prev_angles)/50, decimals=1)
        if int(round(time.time() * 1000)) - stand_tik > 30:
            stand_tik = int(round(time.time() * 1000)) 
            if j < 50:
                current_angles = current_angles + inc_angles
                write_angles(current_angles)
                j+= 1
            elif j == 50:
                i = 0
                prev_angles = current_angles
                prev_state = 1
                flag = 0
                step_height = 17
                loop_time = max_loop_time
                height = 0
                
    # --- Walk ---
    if state == 2 and prev_state != 0:
        prev_state = 2
        
        [T1, T2, T3, T4] = trot_gait(LX,LY,RX, step_height)
        [W1, W2, W3, W4, W5, W6, W7, W8] = walk_gait(LX, LY, RX, step_height)

        if flag == 0:
            stance = T1
        elif flag == 1:
            stance = T2
        elif flag == 2:
            stance = T3
        elif flag == 3:
            stance = T4
       
        angles = np.round(kinematics.solve(stance,kinematics.bodyIK(0,0,0*pi/180,28,height,0)), decimals=1)
        inc_angles = np.round((angles-prev_angles)/7, decimals=1)
        
        if int(round(time.time() * 1000)) - tik > loop_time:
            tik = int(round(time.time() * 1000))
            if LX != 0 or LY != 0 or RX != 0:
                if ((inc_angles[0][2] < 0 and current_angles[0][2] <= angles[0][2]) or (inc_angles[0][2] > 0 and current_angles[0][2] >= angles[0][2])) or \
                ((inc_angles[1][2] < 0 and current_angles[1][2] <= angles[1][2]) or (inc_angles[1][2] > 0 and current_angles[1][2] >= angles[1][2])) or \
                ((inc_angles[2][2] < 0 and current_angles[2][2] <= angles[2][2]) or (inc_angles[2][2] > 0 and current_angles[2][2] >= angles[2][2])) or \
                ((inc_angles[3][2] < 0 and current_angles[3][2] <= angles[3][2]) or (inc_angles[3][2] > 0 and current_angles[3][2] >= angles[3][2])):   
                        write_angles(current_angles)
                        prev_angles = current_angles
                        if flag == 3:
                            flag = 0
                        else:
                            flag += 1
                else:
                    current_angles = current_angles + inc_angles
                    write_angles(current_angles)
            else:
                write_angles(angles = np.round(kinematics.solve(neutral_standing,kinematics.bodyIK(0,0,0*pi/180,28,height,0)), decimals=1))

  
        if int(round(time.time() * 1000)) - led_tik > 100:
            led_tik = int(round(time.time() * 1000))
            pixels[N] = Orange
            pixels[NA] = Orange
            pixels.show()
            pixels.fill(0)
            if N < NUM_PIXELS - 1:
                N += 1
            else:
                N = 0
            
            if NA < NUM_PIXELS - 1:
                NA += 1
            else:
                NA = 0
    # --- Stand/I-Kinematics --- 
    if state == 3 and prev_state !=0:
        prev_state = 3
        roll = np.round(-RX*0.2*pi/180, decimals = 2)
        yaw = np.round(LX*0.16*pi/180, decimals = 2)
        pitch = np.round(LY*0.16*pi/180, decimals = 2)
        if int(round(time.time() * 1000)) - kin_tik > 20:
            kin_tik = int(round(time.time() * 1000))
            angles = np.round(kinematics.solve(neutral_standing,kinematics.bodyIK(roll,yaw,pitch,28,height,0)), decimals=0)
            write_angles(angles)
            
        if int(round(time.time() * 1000)) - led_tik > 100:
            led_tik = int(round(time.time() * 1000))
            pixels[N] = Blue
            pixels[NA] = Blue
            pixels.show()
            pixels.fill(0)
            if N < NUM_PIXELS - 1:
                N += 1
            else:
                N = 0
            
            if NA < NUM_PIXELS - 1:
                NA += 1
            else:
                NA = 0
   

TimeoutError: [Errno 110] Connection timed out

In [ ]:
neutral_resting = np.array([[117,-60,88.5,1], 
                             [117,-60,-88.5,1],
                             [-117,-60,88.5,1],
                             [-117,-60,-88.5,1]])
angles = np.round(kinematics.solve(neutral_resting,kinematics.bodyIK(0,0,0*pi/180,28,0,0)), decimals=1)
print(angles)